In [ ]:
import math
import pandas as pd
import numpy as np
import igraph 
import itertools
import matplotlib.pyplot as plt
from notears_custom import utils
from scipy import stats, linalg
from itertools import chain, combinations, product
from sklearn.linear_model import LinearRegression, Lasso
import math
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from tqdm.notebook import tqdm
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
%matplotlib inline

In [ ]:
# data = pd.read_csv('C:/Users/ehall/Documents/thesis/bayesian_networks/data/rbc.csv')
data = pd.read_csv('C:/Users/ehall/Documents/thesis/bayesian_networks/data/real_data.csv')
# data = data.drop(['eps_g', 'eps_z', 'log_y', 'log_k', 'log_c', 'log_l','log_w','log_i'], axis=1)
data = data.drop(list(['DATE', 'dk']), axis=1)
data.columns = [col.replace(" ", "") for col in data.columns]
data = data.iloc[:,1:]
# data.drop(['rb', 'l', 'n'], axis=1, inplace=True)

# data = data.apply(lambda x: x - x.mean())
# data = data.applymap(lambda x: x + np.random.normal(scale=1))

shift_vars = data.columns.values
shift = data.loc[:,shift_vars].shift()
shift.columns = [str(col) + '_1' for col in shift.columns]
data = pd.concat([data, shift], axis=1)
data = data.iloc[1:,:]

# data = data.iloc[:1000,:]

# data = data[['rm', 'rk', 'z', 'y', 'w', 'u', 'c', 'i', 'g',
#              'rm_1', 'rk_1', 'z_1', 'y_1', 'w_1', 'u_1', 'c_1', 'i_1', 'g_1']]

In [ ]:
class dag():
    
    def __init__(self, m):
        '''
        m is a pandas DataFrame conaining an adjacency matrix
        '''
        self.m = m
        self.nodes = self.m.columns.values
        self.lags = np.array([n for n in self.nodes if '_1' in n])
        if not all(self.m.columns.values == self.m.index.values):
            raise ValueError('Invalid adjacency matrix (columns and rows are not the same)')
        self.directed = self.directed()

            
    def parents(self, n):
        '''
        n is a node in the graph
        '''
        if n not in self.nodes:
            raise ValueError('n is not in graph')
        return self.m.loc[self.m.loc[:,n] != 0, n].index.values
    
    
    def children(self, n):
        '''
        n is a node in the graph
        '''
        if n not in self.nodes:
            raise ValueError('n is not in graph')
        return self.m.loc[:,self.m.loc[n,:] != 0].columns.values
    
    
    def directed(self):
        return utils.is_dag(self.m.values)
        
    
    def depth(self, n):
        '''
        n is a node in the graph
        return the length of the shortest path to a root node
        '''
        if not self.directed:
            raise ValueError('Cannot compute depth, graph is undirected')
        if len(self.parents(n)) == 0:
            return 0
        else:
            return 1 + max([self.depth(p) for p in self.parents(n)])
    
    
    def root_nodes(self):
        return np.array([n for n in self.nodes if len(self.parents(n)) == 0])
        
        
    def isolated_nodes(self):
        return np.array([n for n in d.nodes if (len(d.parents(n)) == 0) & (len(d.children(n)) == 0)])
    
    
    def connected_roots(self):
        return np.array([n for n in d.nodes if (len(d.parents(n)) == 0) & (len(d.children(n)) > 0)])
    
    
    def structure(self):
        M = self.m.copy()
        M[M != 0] = 1
        return M
    
    
    def shd(self, d):
        try:
            return utils.count_accuracy(d.structure().values, self.structure().values)['shd']
        except ValueError as e:
            # Graph is not directed
            return None
    
    
    def impute(self, values):
        na_nodes = values[values.isna()]
        depth = 0
        try:
            max_depth = max([self.depth(n) for n in self.nodes])
        except ValueError as e:
            print('Cannot impute values as graph is not directed')
            raise e
        while depth <= max_depth:
            for node in na_nodes.index:
                if self.depth(node) == depth:
                    values[node] = np.dot(self.m.loc[:,node], values.fillna(0))
            depth += 1
        return values


    def calculate_irf(self, x_0, T=250, verbose=False):
        if verbose:
            print('Simulating irf...')
        for lag in self.lags:
            x_0[lag] = 0
        irf = pd.DataFrame([self.impute(x_0)], columns=self.nodes)
        for t in range(T-1):
            if verbose:
                print('Simulating t={} of {} ({}%)'.format(t+1, T, np.round(100 * ((t+1)/T), decimals=2)))
            nr = pd.Series(np.full(len(self.nodes), np.nan), index=self.nodes)
            for lag in self.lags:
                nr[lag] = irf.iloc[-1,:][lag.rstrip('_1')]
            nr = self.impute(nr)    
            irf = irf.append(nr, ignore_index=True)
        irf.loc[:,'t'] = range(T)
        irf.set_index('t', inplace=True)
        irf.drop(self.lags, axis=1, inplace=True)
        return irf


    def plot_irf(self, irf, layout=None):
        if layout is None:
            side = math.ceil(math.sqrt(len(irf.columns)))
            layout = (side, side)
        axes = irf.plot(subplots=True, layout=layout, 
                        color="black", legend=False)
        for ax, name in zip(axes.flatten(), irf.columns.values):
            ax.axhline(y=0, color="red")
            ax.set_title(name)
        return plt
    
    
    def plot_structure(self):
        M = self.m.values
        g = igraph.Graph.Adjacency((M != 0.0).tolist())
        g.es['weight'] = M[M.nonzero()]
        g.vs['label'] = self.nodes
        g.vs['color'] = 'white'
        g.vs['size'] = 45
        return g

In [ ]:
class roles():
    def __init__(self, exo_states, endo_states, controls, names):
        self.exo_states = exo_states
        self.endo_states = endo_states
        self.controls = controls
        self.exo_states_1 = [x + '_1' for x in self.exo_states]
        self.endo_states_1 = [x + '_1' for x in self.endo_states]
        self.controls_1 = [x + '_1' for x in self.controls]
        self.names = names
        assert len(self.exo_states) + len(self.endo_states) + len(self.controls) == len(names)
        
        self.exo_states_idx = np.where([x in exo_states for x in names])[0]
        self.endo_states_idx = np.where([x in endo_states for x in names])[0]
        self.controls_idx = np.where([x in controls for x in names])[0]
        self.lag_exo_states_idx = np.array([x + len(names) for x in self.exo_states_idx], dtype=int).reshape(-1)
        self.lag_endo_states_idx = np.array([x + len(names) for x in self.endo_states_idx], dtype=int).reshape(-1)
        self.lag_controls_idx = np.array([x + len(names) for x in self.controls_idx], dtype=int).reshape(-1)
        
    
    def lag_idx(self, x):
        return x + len(self.names)

In [ ]:
def lag(n):
    return n + '_1'


def current(n):
    return n.rstrip('_1')

                    
def partial_correlation(y, x, z, data):
    if len(z) == 0:
        pcorr = stats.pearsonr(data[:,y], data[:,x])
    else:
        model_y = LinearRegression(fit_intercept=False, normalize=True)
        model_x = LinearRegression(fit_intercept=False, normalize=True)
        model_y.fit(data[:,z], data[:,y])
        model_x.fit(data[:,z], data[:,x])
        resid_y = data[:,y] - model_y.predict(data[:,z])
        resid_x = data[:,x] - model_x.predict(data[:,z])
        pcorr = stats.pearsonr(resid_y, resid_x) # return p-value
    
    return pcorr


def fisher_z(pcorr, z, data):
    z = math.sqrt(len(data) - len(z) - 3)*math.log((1+pcorr)/(1-pcorr))/2
    pval = 2*(1-stats.norm.cdf(abs(z)))
    return (z, pval)


def potential_states(x, max_states=None):
    limit = len(x) - 1 if max_states is None else max_states + 1
    exo_states = chain.from_iterable(combinations(x, r) for r in range(limit))
    for exo in exo_states:
        y = [z for z in x if z not in exo]
        endo_states = chain.from_iterable(combinations(y, r) for r in range(limit-len(exo)))
        for endo in endo_states:
            controls = [z for z in x if z not in endo and z not in exo]
            yield roles(exo, endo, controls, x)
    return None


def sigma_sq(Y, Y_hat):
    n = Y.shape[0]
    sigma_sq = (1/n)*np.mean(np.sum((Y - Y_hat)**2))
    return sigma_sq


def llik(Y, Y_hat):
    n = Y.shape[0]
    sigma = sigma_sq(Y, Y_hat)
    ll = (-1 * (n/2) * np.log(2*math.pi) - 
               (n/2) * np.log(sigma) - 
               (1/(2*sigma)) * np.sum((Y - Y_hat)**2))
    return ll
    
    
def loglik(roles, data):
    ll = 0

    t_controls = np.append(roles.controls_idx, roles.endo_states_idx)
    n = data.shape[0]
    p = t_controls.shape[0]
    X_hat = np.empty((n,0))
    if len(roles.exo_states) > 0:
        for lag_exo_state, exo_state in zip(roles.exo_states_idx, roles.lag_exo_states_idx):
            X = data[:,lag_exo_state]
            Y = data[:,exo_state]
            model = sm.OLS(Y, X)
            model_fit = model.fit()
            Y_hat = model_fit.predict(X)
            
            ll += llik(Y, Y_hat)
            X_hat = np.append(X_hat.reshape(n,-1), model_fit.predict(X).reshape(n,-1), axis=1)
        
    # X_hat = np.append(X_hat.reshape(n,-1), data[:,roles.lag_endo_states_idx], axis=1)
    X_hat = np.append(data[:,roles.exo_states_idx], data[:,roles.lag_endo_states_idx], axis=1)
    
    if X_hat.shape[1] > 0:
        for control in t_controls:
            Y = data[:,control]
            model = sm.OLS(Y, X_hat)
            model_fit = model.fit()
            Y_hat = model_fit.predict(X_hat)
            ll += llik(Y, Y_hat)
   
    else: # No states in model
        for control in t_controls:
            ll += llik(data[:,control], np.zeros((n,1)))
        
    return ll
        

def aic(L, roles, data):
    k = 2*(len(roles.exo_states) + 
           (len(roles.exo_states) + len(roles.endo_states))*
           (len(roles.controls) + len(roles.endo_states)) )
    n = data.shape[0]
    return 2*k - 2*L

    
def bic(L, roles, data):
    k = 2*(len(roles.exo_states) + 
           (len(roles.exo_states) + len(roles.endo_states))*
           (len(roles.controls) + len(roles.endo_states)) )
    n = data.shape[0]
    return k * np.log(n) - 2 * L


def mse(roles, data):
    t_controls = np.append(roles.controls_idx, roles.endo_states_idx)
    n = data.shape[0]
    p = t_controls.shape[0]
    X_hat = np.empty((n,0))
    if len(roles.exo_states) > 0:
        for lag_exo_state, exo_state in zip(roles.exo_states_idx, roles.lag_exo_states_idx):
            X = data[:,lag_exo_state]
            Y = data[:,exo_state]
            model = sm.OLS(Y, X)
            model_fit = model.fit()
            X_hat = np.append(X_hat.reshape(n,-1), model_fit.predict(X).reshape(n,-1), axis=1) 
    X_hat = np.append(X_hat.reshape(n,-1), data[:,roles.lag_endo_states_idx], axis=1)
    if X_hat.shape[1] > 0:
        X = np.append(data[:,roles.exo_states_idx], data[:,roles.lag_endo_states_idx], axis=1)
        Y = data[:,t_controls]
        model = sm.OLS(Y, X_hat)
        model_fit = model.fit()
        Y_hat = model_fit.predict(X)
        mse = np.mean(np.sum((Y - Y_hat)**2, axis=0))
    else:
        mse = (1/n)*np.sum((data[:,t_controls])**2)
    
    return mse
    
    
def score_tests(roles, data):
    L = loglik(roles, data.values)
    b = bic(L, roles, data.values) 
    a = aic(L, roles, data.values)
    m = mse(roles, data.values)
    tests = {
        'loglik': L,
        'bic': b,
        'aic': a,
        'mse': m
    }  
    return tests


def contraint_tests(roles, names, data):
    tests = []
    # Test that controls and endogenous states are conditionally independent
    # Conditional on lag of endo states and exo states
    for (x, y) in combinations(np.append(roles.controls_idx, roles.endo_states_idx), 2):
        z = np.append(roles.lag_endo_states_idx, roles.exo_states_idx)
        p = partial_correlation(x, y, z, data=data.values)
        pcorr = p[0]
        pval = p[1]
        tests.append({'x': names[x],
                      'y': names[y],
                      'z': names[z],
                      'pcorr': pcorr,
                      'pval': pval})
                    
    # Test that current controls and endo states are independent of 
    # lagged exo states conditional on current exo states
    for (x, y) in itertools.product(np.append(roles.controls_idx, roles.endo_states_idx), roles.exo_states_idx):
        y_1 = roles.lag_idx(y)
        z = np.array([y], dtype=int)
        p = partial_correlation(x, y_1, z, data=data.values)
        pcorr = p[0]
        pval = p[1]
        tests.append({'x': names[x],
                      'y': names[y_1],
                      'z': names[z],
                      'pcorr': pcorr,
                      'pval': pval}) 
    
    # Test that lagged endo states are marginally independent of
    # current exo states
    for (x, y) in itertools.product(roles.lag_endo_states_idx, roles.exo_states_idx):
        p = partial_correlation(x, y, [], data=data.values)
        pcorr = p[0]
        pval = p[1]
        tests.append({'x': names[x],
                      'y': names[y],
                      'z': [],
                      'pcorr': pcorr,
                      'pval': pval}) 
    
    # Test that current controls are independent of their lags
    # conditional on lagged endo states
    for x in roles.controls_idx:
        p = partial_correlation(x, roles.lag_idx(x), roles.lag_endo_states_idx, data=data.values)
        pcorr = p[0]
        pval = p[1]
        tests.append({'x': names[x],
                      'y': names[roles.lag_idx(x)],
                      'z': names[roles.lag_endo_states_idx],
                      'pcorr': pcorr,
                      'pval': pval}) 
    return tests


def evaluate_states(data, roles, tests=['score', 'constraint'], alpha=0.05, verbose=False):
    if verbose: 
        print('Evaluating states {}'.format(list(roles.exo_states) + [es + '_1' for es in roles.endo_states]))
    names = np.array(list(roles.names) + [x + '_1' for x in roles.names]).reshape(-1)
    results = {}
    results['exo_states'] = roles.exo_states
    results['endo_states'] = roles.endo_states
    results['controls'] = roles.controls
    if 'constraint' in tests:
        ct = contraint_tests(roles, names, data) 
        m = len(ct)
        valid = all([test['pval'] > (alpha/2)/m for test in ct])
        if valid and verbose:
            print('Valid states found: {}'.format(np.append(roles.exo_states, roles.endo_states)))
        results['valid'] = valid
        results['mean_pval'] = np.mean([test['pval'] for test in ct])
        results['max_pval'] = max([test['pval'] for test in ct])
        results['min_pval'] = min([test['pval'] for test in ct])
        results['contraint_tests'] = ct
    if 'score' in tests: 
        st = score_tests(roles, data)             
        results = {**results, **st}
    results['nstates'] = len(roles.endo_states) + len(roles.exo_states)
    results['nexo'] = len(roles.exo_states)
    results['nendo'] = len(roles.endo_states)
    results['roles'] = roles
    return results


def choose_states(data, alpha=0.05, tests=['score', 'constraint'], max_states=None, verbose=False):
    results = pd.DataFrame(columns=['valid', 'states', 'mean_pval', 'max_pval', 'min_pval', 'tests'])
    variables = data.columns.values[:np.int64(len(data.columns.values)/2)]
    ps = tqdm(potential_states(variables, max_states=max_states))
    for states in ps:
        result = evaluate_states(data, states, tests, alpha=alpha, verbose=verbose)
        results = results.append(result, ignore_index=True)
    return results

        
def choose_states_parallel(data, alpha=0.05, tests=['score', 'constraints'], max_states=None, verbose=False):
    variables = data.columns.values[:np.int64(len(data.columns.values)/2)]
    states = potential_states(variables, max_states=max_states)
    results = Parallel(n_jobs=cpu_count())(delayed(evaluate_states)(data, state, tests, alpha, verbose) for state in tqdm(states))
    return pd.DataFrame(results)
    
    
def make_adjacency(exo_states, endo_states, controls, data):
    data_names = data.columns.values.tolist()
    data_current = [name for name in data_names if '_1' not in name]
    implied_names = exo_states + endo_states + controls
    
    for name in implied_names:
        if name not in data_names:
            raise ValueError('Name {} missing from data'.format(name))
        if str(name) + '_1' not in data_names:
            raise ValueError('Lag of name {} missing from data'.format(name))
        if any([name not in data_names for name in implied_names]) or any([name not in implied_names for name in data_current]):
            print(data_current)
            print(implied_names)
            raise ValueError('Implied names and data do not align')
    
    names = data_names
    d = len(data.columns)
    result = pd.DataFrame(np.zeros((d, d)), columns=names, index=names)
    
    for exo_state in exo_states:
        model = LinearRegression(fit_intercept=False)
        model.fit(data[[exo_state + '_1']], data[exo_state])
        result.loc[exo_state + '_1', exo_state] = model.coef_[0]
        
    for endo in endo_states + controls:
        regressors = [es + '_1' for es in endo_states] + exo_states
        model = LinearRegression(fit_intercept=False)
        model.fit(data[regressors], data[endo])
        coefs = {}
        for i in range(len(regressors)):
            coefs[regressors[i]] = model.coef_[i]
        for x in regressors:
            result.loc[x, endo] = coefs[x]
    
    return result

In [ ]:
results = choose_states_parallel(data, alpha=0.05, tests=['score'], max_states=6, verbose=True)

In [ ]:
results.sort_values(by='mse', ascending=True)

In [ ]:
results[results['valid']].sort_values(by=['nstates', 'loglik'], ascending=[True, False])

In [ ]:
results.sort_values(by='mean_pval', ascending=False)

In [ ]:
variables = data.columns.values[:int(len(data.columns.values)/2)]
exo_names = ['z', 'g']
endo_names = ['k']
controls = [x for x in variables if x not in exo_names and x not in endo_names]
r = roles(exo_names, endo_names, controls, variables)

print(mse(r, data.values))
print(bic(loglik(r, data.values), r, data.values))
print(loglik(r, data.values))

In [ ]:
# An agnostic way of choosing states from the data
optimal_states = results[results['valid']].sort_values(by=['nstates', 'min_pval'], ascending=[True, False]).iloc[0,:]['states']

In [ ]:
exo_names = list(optimal_states.exo_states)
endo_names = list(optimal_states.endo_states)
controls = optimal_states.controls

print(exo_names)
print(endo_names)
print(controls)

In [ ]:
m = make_adjacency(exo_names, endo_names, controls, data=data)
real_dag = dag(m)

x_0 = pd.Series(np.full(len(real_dag.nodes), np.nan), index=real_dag.nodes)
shock_amt = 1
shock_var = 'z'
if shock_var in exo_names:
    shock_index = np.where(data.columns.values == shock_var)[0][0]
elif shock_var + '_1' in endo_names:
    shock_index = np.where(data.columns.values == shock_var.rstrip('_1'))[0][0]
else:
    raise ValueError('Cannot shock control variable {}'.format(shock_var))

x_0[shock_index] = shock_amt
x_0[int(len(real_dag.nodes)/2):] = 0

real_irf = real_dag.calculate_irf(x_0, T=25)
plt = real_dag.plot_irf(real_irf)
plt.show()

In [ ]:
variables = data.columns.values[:int(len(data.columns.values)/2)]
# Some guess at what the states may be
exo_names = ['z', 'n']
endo_names = ['rm', 'g', 'w', 'u', 'c', 'l']
controls = [x for x in variables if x not in exo_names and x not in endo_names]
r = roles(exo_names, endo_names, controls, variables)

print(exo_names)
print(endo_names)
print(controls)

In [ ]:
m = make_adjacency(exo_names, endo_names, controls, data=data)
real_dag = dag(m)

x_0 = pd.Series(np.full(len(real_dag.nodes), np.nan), index=real_dag.nodes)
shock_amt = 1
shock_var = 'rm'
shock_index = np.where(data.columns.values == shock_var)[0][0]
x_0[shock_index] = shock_amt
x_0[int(len(real_dag.nodes)/2):] = 0

real_irf = real_dag.calculate_irf(x_0, T=25)
plt = real_dag.plot_irf(real_irf)
plt.show()

In [ ]:
variables = data.columns.values[:int(len(data.columns.values)/2)]
# Some guess at what the states may be
exo_names = []
endo_names = []
controls = [x for x in variables if x not in exo_names and x not in endo_names]
r = roles(exo_names, endo_names, controls, variables)

In [ ]:
def bic(L, roles, data):
    # First term is number of slope coefficients, second is number of sigmas
    k = (len(roles.exo_states) + (len(roles.exo_states) + len(roles.endo_states)) * (len(roles.controls) + len(roles.endo_states))) + (len(roles.exo_states) + 1)
    n = data.shape[0]
    return k * np.log(n) - 2 * L

def loglik(roles, data):
    ll = 0
    # Get predicted values of exo_states
    for exo_state, lag_exo_state in zip(roles.exo_states_idx, roles.lag_exo_states_idx):
        X_1 = data[:,lag_exo_state]
        Y_1 = data[:,exo_state]
        model_1 = sm.OLS(Y_1, X_1)
        model_1_fit = model_1.fit()
        ll += model_1.loglike(model_1_fit.params)

    t_controls = np.append(roles.controls_idx, roles.endo_states_idx)
    t_states = np.append(roles.exo_states_idx, roles.lag_endo_states_idx)
    if t_states.shape[0] > 0:
        # Predict current endo states and controls using predicted exo states and past endo states
        X_2 = data[:,t_states]
        Y_2 = data[:,t_controls]
        model_2 = sm.OLS(Y_2, X_2)
        model_2_fit = model_2.fit()
        ll += model_2.loglike(model_2_fit.params)
    else:
        sigma_sq = (1/data.shape[0])*np.sum((data[:,t_controls])**2)
        ll += -1 * (data.shape[0]/2) * np.log(2*math.pi) - (data.shape[0]/2) * np.log(sigma_sq) - (1/(2*sigma_sq)) * np.sum((data[:,t_controls])**2)

    return ll

L = loglik(r, data.values)
b = bic(L, r, data.values)

print(L)
print(b)

In [ ]:
df = pd.DataFrame(columns=['x', 'y', 'pcorr', 'pval(t)'])
for x in list(combinations(controls, 2)):
    df = df.append({'x': x[0],
                    'y': x[1],
                    'z': exo_states + endo_states,
                    'pcorr': partial_correlation(x[0], x[1], exo_states + endo_states, data=data)[0],
                    'pval(t)': partial_correlation(x[0], x[1], exo_states + endo_states, data=data)[1]},
                   ignore_index=True)
df.sort_values(by='pval(t)', ascending=True)

In [ ]:
X = data.loc[:,['g', 'z']]
Y_1 = data.loc[:, 'y']
Y_2 = data.loc[:, 'i']
Y = data.loc[:, ['y', 'i']]

model_1 = sm.OLS(Y_1, X)
model_1_fit = model_1.fit()
model_1_ll = model_1.loglike(model_1_fit.params)

model_2 = sm.OLS(Y_2, X)
model_2_fit = model_2.fit()
model_2_ll = model_2.loglike(model_2_fit.params)

model = sm.OLS(Y, X)
model_fit = model.fit()
model_ll = model.loglike(model_fit.params)

print(model_1_ll + model_2_ll)
print(model_ll)